In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

from model import Schelling

C:\Users\edignum\.virtualenvs\compassproject-OUBZ_QZN\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [ ]:
%%timeit
model = Schelling(width=100, 
                    height=100, density=0.9, 
                    mode='homogeneous', mu1=0.6, std1=0.01,
                    minority_pc=0.5, 
                    window_size=30, conv_threshold=0.0)
for _ in range(100):
    model.step()
# print(model.schedule.steps)  # Show how many steps have actually ran

In [ ]:
model = Schelling(mode='Homogeneous',
            width=80, height=80, 
            density=0.9,
            max_steps=500, 
            minority_pc=0.5, 
            homophily=0.3,
            window_size=30, 
            conv_threshold=0.01,
            torus=True, 
            radius=1,
            mu1=1, 
            std1=1,
            mu2=1, 
            std2=1
        )
model.simulate()
model_out = model.datacollector.get_model_vars_dataframe()

In [ ]:
model_out.head()

In [ ]:
params = {"height": 100, "width": 100, 
          "density": 0.8,
          'move_fraction':0.1,
          "minority_pc": 0.5, 
          'mode':'Homogeneous',
          "mu1": np.arange(0, 0.8, 0.05),
          "std1": [0, 0.02, 0.05, 0.1, 0.2],
          "radius": [1, 2, 3],
#           'mode':'Heterogeneous',
#           'radius':1,
#           'mu1':[0.25, 0.5, 1, 3, 5, 7], 'std1':[0.25, 0.5, 1, 3, 5, 7],
#           'mu2':[0.25, 0.5, 1, 3, 5, 7], 'std2':[0.25, 0.5, 1, 3, 5, 7],
         }

from mesa.batchrunner import batch_run
from multiprocessing import freeze_support
if __name__ == '__main__':
    freeze_support()
    hom_param_sweep = batch_run(
            Schelling,
            parameters=params,
            iterations=5,
            max_steps=200,
            number_processes=None,
            data_collection_period=-1,
            display_progress=True,
    )

In [ ]:
hom_results = pd.DataFrame(hom_param_sweep)
hom_results.head()

g = sns.relplot(
    data=hom_results,
    x="mu1", y="Avg fraction similar",
    hue="radius", col="std1",
    kind="line", col_wrap=3,
    height=4, aspect=1, 
    facet_kws=dict(sharex=False),
)
g.set(ylim=(0.5, 1))
g.fig.subplots_adjust(top=0.9)
g.fig.suptitle('Schelling model, two groups (equal values)', size=20)
plt.show()

In [ ]:
# params = {"height": 50, "width": 50, "density": 0.8, 
#           "minority_pc": 0.5, 
#           'mode':'Heterogeneous',
#           'radius':1,
#           'mu1':[0.25, 0.5, 1, 3, 5, 7], 'std1':[0.25, 0.5, 1, 3, 5, 7],
#           'mu2':[0.25, 0.5, 1, 3, 5, 7], 'std2':[0.25, 0.5, 1, 3, 5, 7],
#          }

# from mesa.batchrunner import batch_run
# from multiprocessing import freeze_support
# if __name__ == '__main__':
#     freeze_support()
#     param_sweep = batch_run(
#             Schelling,
#             parameters=params,
#             iterations=10,
#             max_steps=200,
#             number_processes=None,
#             data_collection_period=-1,
#             display_progress=True,
#     )

In [ ]:
# results = pd.DataFrame(param_sweep)
# results.head()

In [ ]:
# for a_g2 in results['mu2'].unique():
#     for b_g2 in results['std2'].unique():
#         plt.figure(figsize=(10, 5))
#         filtered = results[np.isclose(results['mu2'], a_g2) & np.isclose(results['std2'], b_g2)]
#         _ = sns.lineplot(data=filtered, x="mu1", 
#                          y="Avg fraction similar", hue="std1")
#         plt.suptitle('mu2: ' + str(a_g2) + ', ' + 'std2: ' + str(b_g2))
#         plt.ylim((0.5, 1.))
#         plt.show()